# 1. Adding coordinates to a CSV with addresses

**Prerequisites:**
* AWS SDK for Python ([Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html))
* [Python-dotenv](https://pypi.org/project/python-dotenv/) reads key-value pairs from a .env file and can set them as environment variables.
* [pandas](https://pandas.pydata.org/) for data manipulation

In [15]:
! pip3 install boto3
! pip3 install python-dotenv
! pip3 install pandas

In [16]:
import boto3
from dotenv import load_dotenv
import os
import pandas as pd

In [17]:
# Load environment variables from .env file
load_dotenv()

# Create a client for Amazon Location service
amazon_location_client = boto3.client(
    "location",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name = os.getenv("AWS_REGION"),
)

Now we will use [search_place_index_for_text](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/location/client/search_place_index_for_text.html) function that send a request to [SearchPlaceIndexForText](https://docs.aws.amazon.com/location/latest/APIReference/API_SearchPlaceIndexForText.html)

In [18]:
def amazon_location_geocoder(address):
    result = amazon_location_client.search_place_index_for_text(
        IndexName='MyIndexName', MaxResults=1, Text=address
    )
    if len(result["Results"]) == 0:
        print(f'No results found for: {address}')
        return None
    else:
        top_result = result["Results"][0]
        place = top_result["Place"]
        geometry = place.get("Geometry")
        if geometry:
            longitude = geometry.get("Point", [None, None])[0]
            latitude = geometry.get("Point", [None, None])[1]
        else:
            longitude = None
            latitude = None
        return {
            "street_number": place.get("AddressNumber"),
            "street_name": place.get("Street"),
            "city": place.get("Municipality"),
            "province": place.get("Region"),
            "postal_code": place.get("PostalCode"),
            "country": place.get("Country"),
            "label": place.get("Label"),
            "longitude": longitude,
            "latitude": latitude,
        }

In [19]:
df = pd.read_csv("data/stores/uk_stores_without_coordinates.csv")

for index, fila in df.iterrows():
    address = df.at[index, 'address']
    print(f'Geocoding: {address}')
    result = amazon_location_geocoder(address)
    df.at[index,'longitude'] = result['longitude']
    df.at[index,'latitude'] = result['latitude']

df.to_csv('data/stores/uk_stores_with_coordinates.csv', index=False)

print(f"All stores have been geocoded!")

Geocoding: Station Street, Birmingham, B2 4QA
Geocoding: Station Approach, Bristol, BS1 6QF
Geocoding: St John's Hill, Clapham, Greater London, SW11 2QP
Geocoding: Station Approach, Guildford, GU1 4UT
Geocoding: New Station Street, Leeds, LS1 4DY
Geocoding: Lime Street, Liverpool, L1 1JD
Geocoding: Tooley Street, London, SE1 3QX
Geocoding: Cannon Street, London, EC4N 6AP
Geocoding: The Strand, London, WC2N 5HF
Geocoding: Euston Road, London, NW1 2RT
Geocoding: Euston Road, London, N1 9AL
Geocoding: Bishopsgate, London, EC2M 7PY
Geocoding: Praed Street, London, W2 1HQ
Geocoding: Pancras Road, London, N1C 4QP
Geocoding: Victoria Street, London, SW1V 1JU
Geocoding: Station Approach, London, SE1 8SW
Geocoding: Piccadilly Station Approach, Manchester, M60 7RA
Geocoding: Station Hill, Reading, RG1 1LZ
All stores have been geocoded!
